In [1]:
import pandas as pd
import plotly.graph_objects as go

In [2]:
ma_test_res = pd.read_pickle("ma_test_res.pkl")
all_trades = pd.read_pickle("all_trades.pkl")

In [3]:
all_trades.shape

(67630, 11)

In [4]:
ma_test_res.shape

(630, 8)

In [5]:
ma_test_res.head()

,pair,num_trades,total_gain,mean_gain,min_gain,max_gain,mashort,malong
0,GBP_USD,544,1021.2,1.877206,-103.1,195.1,4,8
1,GBP_USD,311,-339.4,-1.091318,-109.8,253.0,4,16
2,GBP_USD,269,-141.6,-0.526394,-145.6,216.6,8,16
3,GBP_USD,194,-160.3,-0.826289,-132.4,238.4,4,32
4,GBP_USD,151,416.7,2.759603,-133.7,221.6,8,32


In [6]:
ma_test_res.columns

Index(['pair', 'num_trades', 'total_gain', 'mean_gain', 'min_gain', 'max_gain',
       'mashort', 'malong'],
      dtype='object')

In [7]:
ma_test_res = ma_test_res[['pair', 'num_trades', 'total_gain','mashort', 'malong']]

In [8]:
ma_test_res["CROSS"] = "MA_" + ma_test_res.mashort.map(str) + "_" + ma_test_res.malong.map(str)

In [9]:
df_all_gains = ma_test_res.groupby(by=["CROSS", "mashort", "malong"], as_index=False).sum()

In [10]:
df_all_gains.sort_values(by="total_gain", ascending=False, inplace=True)

In [11]:
df_all_gains.head()

,CROSS,mashort,malong,num_trades,total_gain
25,MA_8_16,8,16,5759,5515.2
19,MA_4_8,4,8,11760,3634.5
15,MA_4_16,4,16,6545,3424.4
2,MA_16_32,16,32,2839,-75.2
27,MA_8_32,8,32,3202,-1996.3


In [12]:
ma_8_16 = ma_test_res[ma_test_res.CROSS=='MA_8_16'].copy()

In [13]:
ma_8_16.sort_values(by="total_gain", ascending=False, inplace=True)

In [14]:
total_p = len(ma_8_16.pair.unique())

In [15]:
ma_8_16[ma_8_16.total_gain>0].shape[0]

13

In [16]:
13 / total_p

0.6190476190476191

In [17]:
for cross in df_all_gains.CROSS.unique():
    df_temp = ma_test_res[ma_test_res.CROSS==cross]
    total_p = df_temp.shape[0]
    n_good = df_temp[df_temp.total_gain>0].shape[0]
    print(f"{cross:12} {n_good:4} {(n_good/total_p)*100:4.0f}%")

MA_8_16        13   62%
MA_4_8         15   71%
MA_4_16        13   62%
MA_16_32        9   43%
MA_8_32         8   38%
MA_4_32         8   38%
MA_64_128       7   33%
MA_8_64         5   24%
MA_64_256       4   19%
MA_4_64         6   29%
MA_24_32        4   19%
MA_8_256        3   14%
MA_4_256        4   19%
MA_16_256       0    0%
MA_64_96        3   14%
MA_24_256       1    5%
MA_32_256       0    0%
MA_4_96         6   29%
MA_32_96        4   19%
MA_16_64        4   19%
MA_8_96         5   24%
MA_8_128        3   14%
MA_4_128        4   19%
MA_16_96        3   14%
MA_32_64        4   19%
MA_24_64        4   19%
MA_32_128       3   14%
MA_24_96        2   10%
MA_24_128       2   10%
MA_16_128       1    5%


In [39]:
crosses = df_all_gains.CROSS.unique()[:3]

In [40]:
crosses

array(['MA_8_16', 'MA_4_8', 'MA_4_16'], dtype=object)

In [42]:
ma_test_res.head(2)

,pair,num_trades,total_gain,mashort,malong,CROSS
0,GBP_USD,544,1021.2,4,8,MA_4_8
1,GBP_USD,311,-339.4,4,16,MA_4_16


In [44]:
df_good = ma_test_res[(ma_test_res.CROSS.isin(crosses)) & (ma_test_res.total_gain>0)].copy()

In [47]:
df_good.head(2)

,pair,num_trades,total_gain,mashort,malong,CROSS
0,GBP_USD,544,1021.2,4,8,MA_4_8
30,GBP_CAD,576,317.7,4,8,MA_4_8


In [53]:
our_pairs = list(df_good.pair.value_counts()[:9].index)

In [54]:
our_pairs

['CAD_JPY',
 'EUR_NZD',
 'GBP_JPY',
 'EUR_GBP',
 'EUR_JPY',
 'GBP_CHF',
 'NZD_CHF',
 'NZD_USD',
 'USD_CAD']

In [57]:
all_trades.head(2)

,time,mid_c,DIFF,DIFF_PREV,IS_TRADE,DELTA,GAIN,PAIR,MASHORT,MALONG,DURATION,CROSS
8,2020-06-09 03:00:00+00:00,1.27130,-0.000292,0.000003,-1,-9.1,9.1,GBP_USD,4,8,10.0,MA_4_8
18,2020-06-09 13:00:00+00:00,1.27039,0.000287,-0.001359,1,25.5,25.5,GBP_USD,4,8,8.0,MA_4_8


In [56]:
all_trades["CROSS"] = "MA_" + all_trades.MASHORT.map(str) + "_" + all_trades.MALONG.map(str)

In [58]:
trades_cad_jpy = all_trades[(all_trades.CROSS=="MA_8_16") & (all_trades.PAIR=="CAD_JPY")].copy()

In [61]:
trades_cad_jpy.head()

,time,mid_c,DIFF,DIFF_PREV,IS_TRADE,DELTA,GAIN,PAIR,MASHORT,MALONG,DURATION,CROSS,CUM_GAIN
26,2020-06-09 21:00:00+00:00,80.286,0.037000,-0.002937,1,-9.6,-9.6,CAD_JPY,8,16,9.0,MA_8_16,-9.6
35,2020-06-10 06:00:00+00:00,80.190,-0.005312,0.011625,-1,-119.6,119.6,CAD_JPY,8,16,49.0,MA_8_16,110.0
84,2020-06-12 07:00:00+00:00,78.994,0.054187,-0.010875,1,-0.6,-0.6,CAD_JPY,8,16,12.0,MA_8_16,109.4
96,2020-06-12 19:00:00+00:00,78.988,-0.024938,0.019063,-1,-15.9,15.9,CAD_JPY,8,16,67.0,MA_8_16,125.3
115,2020-06-15 14:00:00+00:00,78.829,0.036875,-0.009062,1,33.3,33.3,CAD_JPY,8,16,20.0,MA_8_16,158.6


In [60]:
trades_cad_jpy['CUM_GAIN'] = trades_cad_jpy.GAIN.cumsum()

In [62]:
def plot_line(df_plot, name):
    fig = go.Figure()
    fig.add_trace(go.Scatter( 
        x=df_plot.time,
        y=df_plot.CUM_GAIN,
        line=dict(width=2,color="#3d825f"),
        line_shape='spline',
        name=name,
        mode='lines'
        ))
    fig.update_layout(width=1000,height=400,
        margin=dict(l=15,r=15,b=10),
        font=dict(size=10,color="#e1e1e1"),
        paper_bgcolor="#1e1e1e",
        plot_bgcolor="#1e1e1e",
        title=name)
    fig.update_xaxes(
        linewidth=1,
        linecolor='#3a4a54',
        showgrid=False,
        zeroline=False
    )
    fig.update_yaxes(
        linewidth=1,
        linecolor='#3a4a54',
        showgrid=False,
        zeroline=False
    )
    fig.show()


In [63]:
plot_line(trades_cad_jpy, "CAD_JPY")

In [64]:
c = 'MA_8_16'
for p in our_pairs:
    temp_df = all_trades[(all_trades.CROSS==c)&(all_trades.PAIR==p)].copy()
    temp_df['CUM_GAIN'] = temp_df.GAIN.cumsum()
    plot_line(temp_df, p + "_" + c)

In [78]:
for c in crosses:   
    temp_df = all_trades[(all_trades.CROSS==c)].copy()    
    temp_df = temp_df.groupby(by="time", as_index=False).sum()
    temp_df['CUM_GAIN'] = temp_df.GAIN.cumsum()
    plot_line(temp_df, c)